## Michigan Data Science Team Lead Water Prediction

This notebook details the preliminary results from a predictive model designed to identify homes in Flint that are at greatest risk of having elevated lead levels in their water. The EPA's threshold for lead exposure in water is 15 ppb, which is used in this model. 

**Note:** as of this writing, the input data are omitted due to possible sensitivities. Some information about this model's features may be inferred from column titles, and I hope to share the full dataset in the future, once said sensitivities are resolved.

Credits:
+ [Jared Webb](https://github.com/jaredawebb/), author of an earlier version of this script
+ [Guangsha Shi](http://www.mse.engin.umich.edu/people/kioup/group/guangsha-shi), coauthor of this script
+ [Alex Chojnacki](https://github.com/the-alex), coauthor of this script


In [1]:
import pandas as pd
import numpy as np


%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sl_df = pd.read_excel("Data/Copy of Service Line Data.xls")
res_df = pd.read_csv("Data/residential_test_data.csv")

In [3]:
merged_df = pd.merge(res_df, sl_df, left_on='PID Dash', right_on='PIDdash',how='left')

### Preprocess features

In this section, we do some light preprocessing, including dropping columns that are problematic for our analysis, and converting certain text variables to numeric factors.

In [4]:

merged_df.rename(columns={'Year Built': 'res_year_built', 'Year_Built': 'sl_year_built'}, inplace=True)


cols_to_drop = ['Sample Number','Date Submitted','Analysis (Lead)','Analysis (Copper)',
               'City','Full Address','Best Address','Clean Address','PID Dash',
               'Property Address','Owner Name','Owner Address','Owner Zip Code',
                'Owner City','Owner State','Owner Country','Tax Payer Name','Tax Payer Address',
                'Tax Payer State','Tax Payer Zip Code','Prop Class',
               'goog_address','Longitude_y', 'Latitude_y', 'Prop_City', 'Prop_State', 'Prop_Zip',
               'Prop_Addr','PIDdash']

cols_to_factorize = ['Street Name','Zip Code','Property Zip Code','Owner Type','Homestead',
                    'Residential Building Style','Rental','Use Type','Old Prop class',
                    'USPS Vacancy','Zoning','Future Landuse','DRAFT Zone','Housing Condition 2012',
                    'Housing Condition 2014','Commercial Condition 2013','Hydrant Type','Class',
                    'Street #']

merged_df.drop(cols_to_drop, axis=1, inplace=True)

for i, row in merged_df.iterrows():
    row['Zip Code'] = str(row['Zip Code'])[:5]

merged_df.replace(to_replace=" ",value="",inplace=True)
merged_df.fillna(value=-999,inplace=True)
# factorize string-based features for XGBoost
for (feat_name, feat_series) in merged_df.iteritems():
    if feat_name in cols_to_factorize:
        merged_df[feat_name], tmp_indexer = pd.factorize(merged_df[feat_name])

### Define target variable

In this exploration, we use the threshold of 15 ppb (based on EPA recommendations) to signify elevated lead levels. Subsequent investigations consider the presence of any lead at all (Lead ppb > 0), as well as extremely elevated levels (e.g. Lead ppb > 100).

In [5]:
# Now that our data is loaded, clean, and ready to go, create numpy arrays to pass
# to a classifier.

Ydata = (merged_df['Lead (ppb)'] > 15)
Xdata = merged_df.drop(['Lead (ppb)', 'Copper (ppb)'], axis=1).values

### Run xgboost classifier with 3-fold cross-validation

In [6]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import StratifiedKFold

from sklearn.metrics import roc_auc_score as roc

class XGBClassifier_new(XGBClassifier):
    def predict(self, X):
        return XGBClassifier.predict_proba(self, X)[:,1]

#cross_validation = StratifiedKFold(Ydata, n_folds=3, shuffle=True,random_state=0)

xgtrain = xgb.DMatrix(Xdata,label=Ydata)
xgmodel = XGBClassifier(n_estimators=1000,seed=1,learning_rate=.03,objective='binary:logistic',nthread=-1)
xgb_param = xgmodel.get_xgb_params()
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=xgmodel.get_params()['n_estimators'],nfold=3,metrics=['auc'],
                 early_stopping_rounds=50, show_progress=False)

Will train until cv error hasn't decreased in 50 rounds.


In [7]:
cvresult

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.618913,0.006659,0.625512,0.001332
1,0.628098,0.006490,0.634233,0.011137
2,0.628056,0.006527,0.634453,0.010999
3,0.635017,0.009724,0.641572,0.009873
4,0.639383,0.015034,0.647719,0.015501
5,0.639383,0.015034,0.647719,0.015501
6,0.639383,0.015034,0.647719,0.015501
7,0.639383,0.015034,0.647719,0.015501
8,0.639383,0.015034,0.647719,0.015501
9,0.643680,0.015616,0.652835,0.017765
